Task 7: Please use Black Friday Sales data you have used for Task 3 and Task 6 for bundling recommendation. This data set does not have product descriptions but product ids so you can work with product ids. You can use notebook n7 as a template.

https://www.kaggle.com/datasets/prepinstaprime/black-friday-sales-data

<a href="https://colab.research.google.com/github/cetinus-gulin/ie423/blob/main/task5-6-7/task7-gulin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize

In [1]:
import pandas as pd

# Load data

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

#dfPurchase = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
dfPurchase = pd.read_csv('train.csv')
dfPurchase.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


# Prepare data

In [3]:
# Drop missing values
dfPurchase.dropna(inplace=True)
dfPurchase.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
13,1000005,P00145042,M,26-35,20,A,1,1,1,2.0,5.0,15665
14,1000006,P00231342,F,51-55,9,A,1,0,5,8.0,14.0,5378
16,1000006,P0096642,F,51-55,9,A,1,0,2,3.0,4.0,13055


In [4]:
# Convert to list format
dfPurchaseList=dfPurchase.groupby('User_ID').Product_ID.apply(list)
dfPurchaseList

User_ID
1000001    [P00248942, P00085942, P00102642, P00110842, P...
1000002    [P00289342, P00034742, P00177442, P00116842, P...
1000003    [P00128042, P00112142, P00182742, P00110742, P...
1000004    [P00184942, P00046742, P00329542, P00114942, P...
1000005    [P00145042, P00324442, P00036842, P00173342, P...
                                 ...                        
1006036    [P00294442, P00118342, P00243942, P00156742, P...
1006037    [P00177442, P00087042, P00025442, P00086442, P...
1006038                    [P00034742, P00086042, P00109542]
1006039    [P00088542, P00254242, P00202742, P00085942, P...
1006040    [P00148642, P00059442, P00024142, P00192042, P...
Name: Product_ID, Length: 5870, dtype: object

# Build model

Associate Rule Mining

In [5]:
# Encode data as transaction matrix
!pip3 install mlxtend --upgrade
from mlxtend.preprocessing import TransactionEncoder

model_trans = TransactionEncoder()
model_trans_arr= model_trans.fit(dfPurchaseList).transform(dfPurchaseList)
dfPurchase = pd.DataFrame(model_trans_arr, columns=model_trans.columns_)
dfPurchase

,P00000142,P00000242,P00000642,P00001042,P00001142,P00001542,P00002142,P00002242,P00003442,P00004242,...,P0096442,P0096542,P0096642,P0096742,P0096842,P0097342,P0099042,P0099742,P0099842,P0099942
0,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,False,False,True,False,True,False,False,False,True,False,...,False,True,False,True,False,True,False,False,False,False
5866,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5867,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5868,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
%%time
# Determine the items and itemsets with at least 1.5% support (generates all itemsets and then filters by support)

from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(dfPurchase, min_support=0.015, use_colnames=True)
frequent_itemsets

CPU times: user 57.3 s, sys: 29.4 s, total: 1min 26s
Wall time: 3min 2s


,support,itemsets
0,0.196252,(P00000142)
1,0.064055,(P00000242)
2,0.087223,(P00000642)
3,0.085690,(P00001042)
4,0.098467,(P00001142)
...,...,...
45922,0.015162,"(P00144642, P00237542, P00242742, P00270942)"
45923,0.016865,"(P00184942, P00237542, P00270942, P00145042)"
45924,0.015162,"(P00184942, P00242742, P00270942, P00145042)"
45925,0.017206,"(P00237542, P00242742, P00270942, P00145042)"


In [10]:
%%time
# Speed up by using pattern fragment growth method for mining frequent itemsets (uses density to find good itemsets)

from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(dfPurchase, min_support=0.015, use_colnames=True)
frequent_itemsets

CPU times: user 9.57 s, sys: 54.4 ms, total: 9.63 s
Wall time: 9.87 s


,support,itemsets
0,0.275128,(P00025442)
1,0.245315,(P00184942)
2,0.239523,(P00059442)
3,0.218228,(P00110842)
4,0.212266,(P00102642)
...,...,...
45922,0.015162,"(P00034742, P00170942)"
45923,0.015673,"(P00002242, P00010742)"
45924,0.015162,"(P00002242, P00057642)"
45925,0.016695,"(P00002242, P00184942)"


In [11]:
# Evaluate the metrics, and filter the items and itemsets that have at least 70% confidence

from mlxtend.frequent_patterns import association_rules

a_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
a_rules.sort_values(by=['confidence'],ascending=False,inplace=True)
a_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
